In [ ]:
# Imports & Setup
import os
import re
import glob
import json
from pathlib import Path

# Google Gemini (Generative AI) SDK
import google.generativeai as genai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# Load API keys from .env
from dotenv import load_dotenv
load_dotenv()

# Set Gemini API key
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Paths
BASE_DIR = Path("../Transcriptions/EMMA_ES-TRANSCRIPTIONS")
KB_DIR = Path("../../KB/chroma_db")
KB_DIR.mkdir(parents=True, exist_ok=True)


In [5]:
# Utility Functions

def clean_text(text: str) -> str:
    """
    Normalize text & remove basic PII (emails, phones, card numbers).
    For production, replace with Presidio or SpaCy NER for stronger PII detection.
    """
    text = text.strip().lower()
    text = re.sub(r"\s+", " ", text)

    # Remove common PII patterns
    text = re.sub(r"\b[\w._%+-]+@[\w.-]+\.\w{2,}\b", "[EMAIL]", text)  # emails
    text = re.sub(r"\b\d{10,16}\b", "[NUMBER]", text)  # phone/credit card numbers
    text = re.sub(r"\b\d{3}[-.\s]?\d{2}[-.\s]?\d{4}\b", "[ID]", text)  # SSN-like patterns

    return text


def load_transcripts(path: Path):
    """
    Load all transcript .txt files from folder
    """
    files = glob.glob(str(path / "*.txt"))
    transcripts = []
    for f in files:
        with open(f, "r", encoding="utf-8") as file:
            transcripts.append({"file": os.path.basename(f), "text": file.read()})
    return transcripts

In [6]:
# Load & Clean Transcripts

raw_data = load_transcripts(BASE_DIR)

print(f"Loaded {len(raw_data)} transcripts")

# Clean
for doc in raw_data:
    doc["clean_text"] = clean_text(doc["text"])

print(raw_data[0]["clean_text"][:500]) 

Loaded 16 transcripts
¿qué tal? aquí le dije más colchones. ¿cómo le puedo ayudar? ah, buenas. oiga, este, mire, quería comprar un colchón individual, pero el essential, o no sé cómo se le llama, ¿no? sí, el lema essential. ajá, entonces lo iba a hacer por la red, pero vi que el sistema de pago es paypal. entonces quería saber si hay alguna otra forma de pago que no sea paypal, que me cae gordo. ay, a ver, a ver, a ver. quiero comprar un colchón essential, pero por la red, o sea, cuando me metí a la página de ustedes


In [7]:
# Chunking

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len,
)

docs = []
for doc in raw_data:
    chunks = text_splitter.split_text(doc["clean_text"])
    for i, chunk in enumerate(chunks):
        docs.append({
            "id": f"{doc['file']}_chunk{i}",
            "text": chunk,
            "metadata": {"source": doc["file"], "chunk_id": i}
        })

print(f"Generated {len(docs)} chunks from transcripts")

Generated 40 chunks from transcripts


In [8]:
# Embeddings + Vector DB (using SentenceTransformers)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Choose a multilingual embedding model (works with Spanish + English)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v1"
)

# Initialize Chroma vector DB
vectorstore = Chroma(
    collection_name="transcripts_kb",
    embedding_function=embedding_model,
    persist_directory=str(KB_DIR)
)

# Add docs into Chroma
texts = [d["text"] for d in docs]
metadatas = [d["metadata"] for d in docs]
ids = [d["id"] for d in docs]

vectorstore.add_texts(texts=texts, metadatas=metadatas, ids=ids)
vectorstore.persist()

print("✅ Knowledge Base updated & stored in KB/chroma_db/ with SentenceTransformers")

C:\Users\user\AppData\Local\Temp\ipykernel_2148\3889173006.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
d:\AI-Calling-Agent\aenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARN

✅ Knowledge Base updated & stored in KB/chroma_db/ with SentenceTransformers


C:\Users\user\AppData\Local\Temp\ipykernel_2148\3889173006.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [9]:
# Quick Retrieval Test

query = "How does payment system works using Paypal at agency?"
results = vectorstore.similarity_search(query, k=5)

for r in results:
    print("----")
    print("Source:", r.metadata["source"])
    print(r.page_content[:300])

----
Source: 2020_12_21_LUFR_529988843072___EMMA_ES__.txt
en mercado libre, ya lo entiendo ahora. o sea, no estoy hablando directamente a la fábrica o a la cliente. perdón, me hice entender mal. o sea, nosotros también nos encontramos en mercado libre. ajá, ok. entonces, ajá, quería ver si solo se puede pagar con paypal o si se puede pagar con tarjeta de d
----
Source: 2020_12_21_LUFR_529988843072___EMMA_ES__.txt
¿qué tal? aquí le dije más colchones. ¿cómo le puedo ayudar? ah, buenas. oiga, este, mire, quería comprar un colchón individual, pero el essential, o no sé cómo se le llama, ¿no? sí, el lema essential. ajá, entonces lo iba a hacer por la red, pero vi que el sistema de pago es paypal. entonces quería
----
Source: 2020_12_21_LUFR_529988843072___EMMA_ES__.txt
tarjeta de crédito acepta de visa, american express, mastercard, tarjetas de débito, también con mastercard débito, visa de débito, mercado pago. tiene más métodos de pago que serían efectivo. oxo, pagas y sacrajetas interna